<a href="https://colab.research.google.com/github/tr3nt-tayl0r/CIMIS/blob/main/cimis_corrections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone into github repo locally

In [ ]:
!git clone https://github.com/tr3nt-tayl0r/CIMIS_data.git

fatal: destination path 'CIMIS_data' already exists and is not an empty directory.


In [ ]:
import glob
import os
import sys
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import json
import requests
from tables import NaturalNameWarning
import warnings
warnings.filterwarnings('ignore',category = NaturalNameWarning)
verbose = False
import datetime
import matplotlib.pyplot as plt

In [ ]:
def fix_col_names(df):
  '''corrects column names from CIMIS based on dictionary'''
  rename_dict = {'DayAirTmpMin.Value':'Tmin',
               'DayAirTmpMax.Value':'Tmax',
               'DayDewPnt.Value':'Tdew',
               'DayAirTmpAvg.Value':'Tavg',
               'DayEto.Value':'ETo',
               'DayRelHumMin.Value':'RHmin',
               'DayRelHumAvg.Value':'RHavg',
               'DayPrecip.Value':'Pr'}
  df.rename(columns=rename_dict, inplace=True)
  dfout = df[['Date','Tmin','Tmax','Tdew','Tavg','ETo','Pr','RHmin','RHavg']]
  dfout.set_index(pd.to_datetime(dfout.Date),inplace=True)
  return dfout

def corr_nref(df,tminNRef='Tmin',tdewNRef='Tdew',tmaxNRef='Tmax',bT=True):
    df['dT']=df[tminNRef]-df[tdewNRef]
    if bT==True:
        df['bT']=0.3
    dfAI = df[['Pr','ETo']].dropna(axis=0)
    AI = dfAI['Pr'].mean()/df['ETo'].mean()
    print('\t\tAI = '+str(np.round(AI,2)))

    if AI<=0.05:
        aT=5
    elif (AI >0.05) & (AI<=0.2):
        aT=2.5
    elif (AI >0.2) & (AI<=0.5):
        aT=1.5
    elif (AI >0.5) & (AI<=0.65):
        aT=0.5
    else:
        aT=0

    df['aT']=aT

    # Might need to add a line only to correct if dT > aT from the logic statements above
    for dT in df['dT']:
      if dT > aT:
        df[tmaxNRef+'_corr2']= df[tmaxNRef]-df.bT*(df.dT-df.aT) #(2.13)
        df[tminNRef+'_corr2']= df[tminNRef]-df.bT*(df.dT-df.aT) #(2.14)
        df[tdewNRef+'_corr2']= df[tdewNRef]+(1.0-df.bT)*(df.dT-df.aT) #(2.15)]
      else:
        df[tmaxNRef+'_corr2']= df[tmaxNRef]
        df[tminNRef+'_corr2']= df[tminNRef]
        df[tdewNRef+'_corr2']= df[tdewNRef]
    df['AI']= AI
    # df.set_index(df.Date,inplace=True)
    return df